In [1]:
import pandas as pd
import math
import time
import spacy
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize  
import json
import numpy as np
import torch
import torch.nn as nn
import torchvision
from torchvision import datasets, transforms
import matplotlib.pyplot as plt

ModuleNotFoundError: No module named 'spacy'

## Load the data after it has been preprocessed with the sentiment classifier

In [9]:
train = np.load("data/training_v1.npy",allow_pickle = True)
test = np.load("data/evaluation_v1.npy",allow_pickle = True)

In [12]:
data_eval = pd.DataFrame(test,columns = ['id','timestamp','retweet_count','user_verified','user_statuses_count','user_followers_count','user_friends_count','user_mentions','urls','hashtags','text','sentiment'])
data = pd.DataFrame(train,columns = ['id','timestamp','user_verified','user_statuses_count','user_followers_count','user_friends_count','user_mentions','urls','hashtags','text','sentiment'])
nlp = spacy.load('en_core_web_sm')

,id,timestamp,user_verified,user_statuses_count,user_followers_count,user_friends_count,user_mentions,urls,hashtags,text,sentiment
0,665776,1588644489777,False,229,36,99,NaN,NaN,NaN,"Coronavirus, no spring break, Chris Brown fan ...",0
1,665777,1588463988413,True,106991,41273,326,NaN,NaN,NaN,This fits the data.,1
2,665778,1588563932106,False,24127,345,200,NaN,NaN,NaN,Y’all talk about the Coronavirus like y’all be...,2
3,665779,1588639210045,True,771,6555,83,NaN,twitter.com/i/web/status/1…,NaN,"As this coronavirus and covid-19 go on, the fo...",3
4,665780,1588692867357,False,32595,6490,1206,NaN,vice.com/en_us/article/…,NaN,Big Brother is not very good at watching you. ...,4
...,...,...,...,...,...,...,...,...,...,...,...
285329,951105,1588673869339,True,10532,6506208,119,NaN,twitter.com/i/web/status/1…,"IndianArmy, HarKaamDeshKeNaam, Soldier",#IndianArmy fights COVID-19\n\n#HarKaamDeshKeN...,285329
285330,951106,1588296137844,False,151,105,100,NaN,twitter.com/i/web/status/1…,NaN,(04/22/2020) “CUT-THE-RONA” \n\nWe would like ...,285330
285331,951107,1588550324249,False,381,77,210,NaN,NaN,NaN,It's just starting.,285331
285332,951108,1588656171612,True,33982,974249,340,NaN,NaN,NaN,It is in no way a surprise that one of the fou...,285332


## Verifier user

Boolean to 0 or 1

In [3]:
#replace the boolean by its int value
df=pd.DataFrame(data_eval["user_verified"])
data_eval["user_verified"]=df.applymap(lambda x: int(x))

df=pd.DataFrame(data["user_verified"])
data["user_verified"]=df.applymap(lambda x: int(x))

## Tweet with # and number of #

In [4]:
def get_hashtag(x) :
    x = str(x) 
    if "#" in x :
        return 1
    return 0

def get_number_hashtag(x) :
    x = str(x)
    count = 0
    for k in x :
        if k == "#" :
            count+=1
    return count

df=pd.DataFrame(data["text"])
data["num_hashtag"] = df.applymap(lambda x: get_number_hashtag(x))
data["got_hashtag"] = df.applymap(lambda x: get_hashtag(x))

df=pd.DataFrame(data_eval["text"])
data_eval["num_hashtag"] = df.applymap(lambda x: get_number_hashtag(x))
data_eval["got_hashtag"] = df.applymap(lambda x: get_hashtag(x))

## Tweet with @ and number of

In [5]:
def get_at(x) :
    x = str(x) 
    if "@" in x :
        return 1
    return 0

def get_number_at(x) :
    x = str(x)
    count = 0
    for k in x :
        if k == "@" :
            count+=1
    return count

df=pd.DataFrame(data["text"])
data["num_at"] = df.applymap(lambda x: get_number_at(x))
data["got_at"] = df.applymap(lambda x: get_at(x))

df=pd.DataFrame(data_eval["text"])
data_eval["num_at"] = df.applymap(lambda x: get_number_at(x))
data_eval["got_at"] = df.applymap(lambda x: get_at(x))

## Tweet with links and number of

In [6]:
def get_link(x) :
    x = str(x) 
    if "https" in x :
        return 1
    return 0

def get_number_link(x) :
    x = str(x)
    count = x.count("https")
    return count

df=pd.DataFrame(data["text"])
data["num_link"] = df.applymap(lambda x: get_number_link(x))
data["got_link"] = df.applymap(lambda x: get_link(x))

df=pd.DataFrame(data_eval["text"])
data_eval["num_link"] = df.applymap(lambda x: get_number_link(x))
data_eval["got_link"] = df.applymap(lambda x: get_link(x))

## Tweet length

In [7]:
def get_lenght(x) :
    return len(str(x))

df=pd.DataFrame(data["text"])
data["length"] = df.applymap(lambda x: get_lenght(x))

df=pd.DataFrame(data_eval["text"])
data_eval["length"] = df.applymap(lambda x: get_lenght(x))

## Contains RT or similar

In [8]:
def contains_RT(x) :
    liste = ["rt","retweet"]
    x = str(x).lower()
    for k in liste :
        if k in x :
            return 1
    return 0

df=pd.DataFrame(data["text"])
data["contains_rt"] = df.applymap(lambda x: contains_RT(x))

df=pd.DataFrame(data_eval["text"])
data_eval["contains_rt"] = df.applymap(lambda x: contains_RT(x))

## Followers, friends and status count

Average and center these data.

In [9]:
cols=["user_statuses_count","user_followers_count","user_friends_count","length"]
for c in cols:
    df=pd.DataFrame(data[c])
    m=data[c].mean()
    v=math.sqrt(data[c].var())
    data[c]=df.applymap(lambda x: (x-m)/v)
    
    # the same mean and sigma
    df=pd.DataFrame(data_eval[c])
    data_eval[c]=df.applymap(lambda x: (x-m)/v)

## Posted at different times

In [10]:
def weak(timestamp) :
    if int(time.ctime(int(str(timestamp)[:-3]))[11:13]) in [7,8,9,10,11,12] :
        return 1
    return 0

def strong(timestamp) :
    if int(time.ctime(int(str(timestamp)[:-3]))[11:13]) in [0,1] :
        return 1
    return 0

def zeros_pic(timestamp) :
    if int(time.ctime(int(str(timestamp)[:-3]))[11:13]) in [17,18,19,20,21,22,23] :
        return 1
    return 0

df=pd.DataFrame(data["timestamp"])
data["weak"] = df.applymap(lambda x: weak(x))
data["strong"] = df.applymap(lambda x: strong(x))
data["zeros_pic"] = df.applymap(lambda x: zeros_pic(x))

df=pd.DataFrame(data_eval["timestamp"])
data_eval["weak"] = df.applymap(lambda x: weak(x))
data_eval["strong"] = df.applymap(lambda x: strong(x))
data_eval["zeros_pic"] = df.applymap(lambda x: zeros_pic(x))

## Some words are upper

In [11]:
def is_upper(x) :
    x = str(x)
    x = x.split(" ")
    for k in x :
        if k.isupper() :
            return 1
    return 0

df=pd.DataFrame(data["text"])
data["is_upper"] = df.applymap(lambda x: is_upper(x))

df=pd.DataFrame(data_eval["text"])
data_eval["is_upper"] = df.applymap(lambda x: is_upper(x))

## Contains ! 

In [12]:
def contains_excl(x) :
    if "!" in str(x) :
        return 1
    return 0

df=pd.DataFrame(data["text"])
data["contains_excl"] = df.applymap(lambda x: contains_excl(x))

df=pd.DataFrame(data_eval["text"])
data_eval["contains_excl"] = df.applymap(lambda x: contains_excl(x))

## Contains organisation or location or person

In [61]:
passage = [-1] 
def contains_per_org_loc(x):
    x = nlp(x)
    per = 0
    org = 0
    gpe = 0
    passage[0] +=1
    if passage[0]%10000 == 0 :
        print(passage[0])
    for k in x.ents :
        if k.label_ == "PERSON" :
            per = 1
        if k.label_ == "ORG" :
            org = 1
        if k.label_ == "GPE" :
            gpe = 1
    return str(per) + " " + str(org) + " " + str(gpe)


df=pd.DataFrame(data["text"])
data["contains_per_org_loc"] = df.applymap(lambda x: contains_per_org_loc(x))

df=pd.DataFrame(data_eval["text"])
data_eval["contains_per_org_loc"] = df.applymap(lambda x: contains_per_org_loc(x))

0


KeyboardInterrupt: 

In [ ]:
def contains_per(x):
    return str(x).split(" ")[0]

def contains_org(x):
    return str(x).split(" ")[1]

def contains_gpe(x):
    return str(x).split(" ")[2]

df=pd.DataFrame(data["contains_per_org_loc"])
data["contains_per"] = df.applymap(lambda x: contains_per(x))
data["contains_org"] = df.applymap(lambda x: contains_org(x))
data["contains_gpe"] = df.applymap(lambda x: contains_gpe(x))

df=pd.DataFrame(data_eval["contains_per_org_loc"])
data_eval["contains_per"] = df.applymap(lambda x: contains_per(x))
data_eval["contains_org"] = df.applymap(lambda x: contains_org(x))
data_eval["contains_gpe"] = df.applymap(lambda x: contains_gpe(x))

## Outliers : the accounts

Some account are really suspicious and after normalisation and center, contains more than 47 fore some features. We will look closely.

In [ ]:
followers = data.copy()

In [ ]:
followers = followers.sort_values(by=["user_followers_count"],ascending=False)

### Above 47 
There are only a few tweets, and the best value to predict is 40000 retweets.

### Between 30 and 47 
Best retweet value is 17000.

### Below 30 
The retweet numbers are not that high constantly. It is not relevent to predict a constant value there.

In [ ]:
S = 0 # error if we predict 0
e = 0 # error if we predict 17 000
for k in range(followers.shape[0]) :
    tweet = followers.iloc[k]
    # here between 30 and 47
    if tweet.user_followers_count > 30  : 
        if tweet.user_followers_count < 47 :
            S += tweet.retweet_count
            e+= abs(tweet.retweet_count-17000)
            # show the tweets
            #print(tweet.user_followers_count,tweet.retweet_count)
    else :
        break
#print(S,e)

## Drop the rest

In [ ]:
to_drop = ["timestamp","user_mentions","urls","hashtags","text"]
data = data.drop(columns = to_drop)
data_eval = data_eval.drop(columns = to_drop)

## Save the data

In [22]:
data.to_csv("data/training_process.csv")
data_eval.to_csv("data/evaluation_process.csv")